엑셀데이터를 사용하지 않고

상표, 주유소명, 주소, 유종별 평균가격(보통휘발유, 경유), 부가정보(세차장 여부, 충전소여부, 경정비 여부, 편의점 여부, 24시 영업 여부)를 가져와보자.(서울시 구별)

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [24]:
url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver-linux64/chromedriver"))
driver.get(url)


In [6]:
driver.refresh()
driver.maximize_window()

In [7]:
from selenium.webdriver.common.by import By

sido = driver.find_element(By.ID, "SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME, "option")

for idx, sido_name in enumerate(sido_list):
    print(str(idx) + "," + sido_name.get_attribute("value"))



0,
1,서울특별시
2,부산광역시
3,대구광역시
4,인천광역시
5,광주광역시
6,대전광역시
7,울산광역시
8,세종특별자치시
9,경기도
10,강원특별자치도
11,충청북도
12,충청남도
13,전북특별자치도
14,전라남도
15,경상북도
16,경상남도
17,제주특별자치도


In [8]:
sidonames = []
for idx, sido_name in enumerate(sido_list):
    if(sido_name.get_attribute("value") != ''):
        sidonames.append(sido_name.get_attribute("value"))
print(sidonames)
print(len(sidonames))


['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
17


In [9]:
sigungu = driver.find_element(By.ID, "SIGUNGU_NM0")
sigungu_list = sigungu.find_elements(By.TAG_NAME, "option")

for idx, sigungu_name in enumerate(sigungu_list):
    print(str(idx) + "," + sigungu_name.get_attribute("value"))


0,
1,강남구
2,강동구
3,강북구
4,강서구
5,관악구
6,광진구
7,구로구
8,금천구
9,노원구
10,도봉구
11,동대문구
12,동작구
13,마포구
14,서대문구
15,서초구
16,성동구
17,성북구
18,송파구
19,양천구
20,영등포구
21,용산구
22,은평구
23,종로구
24,중구
25,중랑구


In [10]:
sigungu_names = []
for idx, sigungu_name in enumerate(sigungu_list):
    if(sigungu_name.get_attribute("value") != ''):
        sigungu_names.append(sigungu_name.get_attribute("value"))
print(sigungu_names)
print(len(sigungu_names))

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
25


In [11]:
print(sidonames)
print(sigungu_names)

['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [12]:
#초기 검색 체크
search_clear_type1 = driver.find_element(By.CLASS_NAME, "check_type1")
NORM_click = search_clear_type1.find_elements(By.CLASS_NAME, "input_check")

for i in range(len(NORM_click)):
    if not NORM_click[i].is_selected():
        NORM_click[i].click()

In [13]:
#상표 전체 체크.

entireiconcheck = driver.find_element(By.XPATH, """//*[@id="POLL_DIV_ALL"]""")

entireiconcheck.click()
print(entireiconcheck.get_attribute("class")) #모두 체크되어 있다면 class는 btn_type2_sel로 되고, 하나라도 해제되면 btn_type2
#따라서 한번 더 클릭해 초기 세팅을 마친다.
if entireiconcheck.get_attribute("class") == "btn_type2":
    entireiconcheck.click()

btn_type2


In [14]:
#부가기능 체크
search_clear_type1_img = driver.find_elements(By.CLASS_NAME, "check_type1_img")
conv_click = search_clear_type1_img[1].find_elements(By.CLASS_NAME, "input_check")
conv_click[0].text

for i in range(len(conv_click)):
    if not conv_click[i].is_selected():
        conv_click[i].click()

In [15]:
obicon = []
obname = []
obloc = []
obhprice = []
obgprice = []
obclean = []
obchge =  []
obmech = []
obconv = []
ob24h = []

In [19]:
import time
from tqdm.notebook import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

gubyul_df = []

for sigungu_name in tqdm(sigungu_names[:]):

    sigungu = driver.find_element(By.ID, "SIGUNGU_NM0")
    sigungu.send_keys(sigungu_name)
    driver.implicitly_wait(3)#이게 훨 빠름.

    
    #바꾸기

    obicon = []
    obname = []
    obloc = []
    obhprice = []
    observicesetlist = []

    g_obicon = []
    g_obname = []
    g_obloc = []
    g_obgprice = []
    g_observicesetlist = []

    obgu = []
    obsi = []
    

    body1 = driver.find_element(By.ID , 'os_price1')
    icount = driver.find_element(By.ID, "totCnt")
    #print(int(icount.text))

    #보통휘발유 클릭
    obhui = driver.find_element(By.ID, "os_layer2")
    obhui.text
    obhui.click()

    for i in range(1, int(icount.text)+1):
        huilist = body1.find_element(By.XPATH, """//*[@id="body1"]/tr[{}]/td[1]/a""".format(i))

        huilist.click()

        up = driver.find_element(By.XPATH, """//*[@id="map_div"]/div[5]/div[1]/div/div/div[2]""")

        webdriver.ActionChains(driver).click_and_hold(up).pause(0.45).release().perform()


        #주소
        box=driver.find_element(By.ID, "rd_addr")
        addrreal = box.text.split('\n')
        addrtext = addrreal[0]
        obloc.append(addrtext)

        #상표
        soilbox=driver.find_element(By.ID, "poll_div_nm")
        soilboxtext = soilbox.text
        obicon.append(soilboxtext)

        #이름
        namebox=driver.find_element(By.ID, "os_nm")
        #print(namebox.text)
        nameboxtext = namebox.text
        obname.append(nameboxtext)
        #print("{} 완료".format(nameboxtext))

        #보통휘발유 가격
        huipricebox = driver.find_element(By.ID, "b027_p")
        huipriceboxtext = huipricebox.text
        huipriceboxtext=int(huipriceboxtext.replace(',', ''))
        #print(huipriceboxtext)
        obhprice.append(huipriceboxtext)

        #부가정보 모음

        servicebox = driver.find_element(By.CLASS_NAME, "service")
        service_list = servicebox.find_elements(By.TAG_NAME, "img")
        svreallist = set([])
        for idx in range(len(service_list)):
            svreallist.add(service_list[idx].get_attribute("alt"))
        #print(svreallist)
        observicesetlist.append(svreallist)
        #print(len(observicesetlist), observicesetlist)

        #부가정보 세차장

        #부가정보 충전소

        #부가정보 경정비

        #부가정보 편의점

        #부가정보 24시
    print("{} 휘발유 완료".format(sigungu_name))
    body2 = driver.find_element(By.ID , 'os_price3')
    icount = driver.find_element(By.ID, "totCnt")

    #경유 클릭
    obgasol = driver.find_element(By.ID, "os_layer3")
    obgasol.text
    obgasol.click()

    for i in range(1, int(icount.text)+1):
        g_huilist = body2.find_element(By.XPATH, """//*[@id="body3"]/tr[{}]/td[1]/a""".format(i))
        g_huilist.click()
        webdriver.ActionChains(driver).click_and_hold(up).pause(0.45).release().perform()

        #g주소
        g_box=driver.find_element(By.ID, "rd_addr")
        g_addrreal = g_box.text.split('\n')
        g_addrtext = g_addrreal[0]
        g_obloc.append(g_addrtext)

        #g상표
        g_soilbox=driver.find_element(By.ID, "poll_div_nm")
        g_soilboxtext = g_soilbox.text
        g_obicon.append(g_soilboxtext)

        #g이름
        g_namebox=driver.find_element(By.ID, "os_nm")
        #print(namebox.text)
        g_nameboxtext = g_namebox.text
        g_obname.append(nameboxtext)

        #g경유 가격
        g_huipricebox = driver.find_element(By.ID, "d047_p")
        g_huipriceboxtext = g_huipricebox.text
        g_huipriceboxtext=int(g_huipriceboxtext.replace(',', ''))
        #print(huipriceboxtext)
        g_obgprice.append(g_huipriceboxtext)

        #g부가정보 모음

        g_servicebox = driver.find_element(By.CLASS_NAME, "service")
        g_service_list = g_servicebox.find_elements(By.TAG_NAME, "img")
        g_svreallist = set([])
        for idx in range(len(g_service_list)):
            g_svreallist.add(g_service_list[idx].get_attribute("alt"))
        #print(svreallist)
        g_observicesetlist.append(g_svreallist)

    huidf = pd.DataFrame(index = obname)
    #huidf['obname'] = obname
    huidf['obloc'] = obloc
    huidf['obicon'] = obicon
    huidf['obhprice'] = obhprice
    huidf['observicesetlist'] = observicesetlist

    g_huidf = pd.DataFrame(index = obname)
    g_huidf['g_obgprice'] = g_obgprice

    concat_oildf = pd.concat([huidf, g_huidf], axis = 1)
    concat_oildf = concat_oildf[['obloc', 'obicon', 'obhprice', 'g_obgprice', 'observicesetlist']]
    concat_oildf

    for gudf in list(concat_oildf['obloc'].values):
        gudf = gudf.split(' ')
        obsi.append(gudf[0])
        obgu.append(gudf[1])
    
    concat_oildf['si_list'] = obsi
    concat_oildf['gu_list'] = obgu

    concat_oildf

    concat_oildf

    gubyul_df.append(concat_oildf)
    print("{} 경유 완료".format(sigungu_name))    

print(len(gubyul_df), gubyul_df)

  0%|          | 0/25 [00:00<?, ?it/s]

강남구 휘발유 완료
강남구 경유 완료
강동구 휘발유 완료
강동구 경유 완료
강북구 휘발유 완료
강북구 경유 완료
강서구 휘발유 완료
강서구 경유 완료
관악구 휘발유 완료
관악구 경유 완료
광진구 휘발유 완료
광진구 경유 완료
구로구 휘발유 완료
구로구 경유 완료
금천구 휘발유 완료
금천구 경유 완료
노원구 휘발유 완료
노원구 경유 완료
도봉구 휘발유 완료
도봉구 경유 완료
동대문구 휘발유 완료
동대문구 경유 완료
동작구 휘발유 완료
동작구 경유 완료
마포구 휘발유 완료
마포구 경유 완료
서대문구 휘발유 완료
서대문구 경유 완료
서초구 휘발유 완료
서초구 경유 완료
성동구 휘발유 완료
성동구 경유 완료
성북구 휘발유 완료
성북구 경유 완료
송파구 휘발유 완료
송파구 경유 완료
양천구 휘발유 완료
양천구 경유 완료
영등포구 휘발유 완료
영등포구 경유 완료
용산구 휘발유 완료
용산구 경유 완료
은평구 휘발유 완료
은평구 경유 완료
종로구 휘발유 완료
종로구 경유 완료
중구 휘발유 완료
중구 경유 완료
중랑구 휘발유 완료
중랑구 경유 완료
25 [                                        obloc    obicon  obhprice  g_obgprice  \
(주)보성 세곡주유소              서울 강남구 헌릉로 731 (세곡동)     SK에너지      1648        1519   
방죽주유소                   서울 강남구 밤고개로 215 (율현동)     GS칼텍스      1677        1529   
HD현대오일뱅크㈜직영 산성셀프주유소            서울 강남구 헌릉로 730  HD현대오일뱅크      1677        1539   
자곡셀프주유소                 서울 강남구 밤고개로 120 (자곡동)     SK에너지      1697        1559   
(주)중앙에너비스 수서지점           서울 강남구 광평로 202 (수서동)     SK에너지      1

In [18]:
import time
from tqdm.notebook import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

gubyul_df = []
#sigungu = driver.find_element(By.ID, "SIGUNGU_NM0")
#sigungu.send_keys(sigungu_name)
#driver.implicitly_wait(3)#이게 훨 빠름.
#
#바꾸기
obicon = []
obname = []
obloc = []
obhprice = []
observicesetlist = []
g_obicon = []
g_obname = []
g_obloc = []
g_obgprice = []
g_observicesetlist = []
obgu = []
obsi = []

body1 = driver.find_element(By.ID , 'os_price1')
icount = driver.find_element(By.ID, "totCnt")
#print(int(icount.text))
#보통휘발유 클릭
obhui = driver.find_element(By.ID, "os_layer2")
obhui.text
obhui.click()
for i in range(1, int(icount.text)+1):
    huilist = body1.find_element(By.XPATH, """//*[@id="body1"]/tr[{}]/td[1]/a""".format(i))
    huilist.click()
    up = driver.find_element(By.XPATH, """//*[@id="map_div"]/div[5]/div[1]/div/div/div[2]""")
    webdriver.ActionChains(driver).click_and_hold(up).pause(0.45).release().perform()
    #주소
    box=driver.find_element(By.ID, "rd_addr")
    addrreal = box.text.split('\n')
    addrtext = addrreal[0]
    obloc.append(addrtext)
    #상표
    soilbox=driver.find_element(By.ID, "poll_div_nm")
    soilboxtext = soilbox.text
    obicon.append(soilboxtext)
    #이름
    namebox=driver.find_element(By.ID, "os_nm")
    #print(namebox.text)
    nameboxtext = namebox.text
    obname.append(nameboxtext)
    print("{} 완료".format(nameboxtext))
    #보통휘발유 가격
    huipricebox = driver.find_element(By.ID, "b027_p")
    huipriceboxtext = huipricebox.text
    huipriceboxtext=int(huipriceboxtext.replace(',', ''))
    #print(huipriceboxtext)
    obhprice.append(huipriceboxtext)
    #부가정보 모음
    servicebox = driver.find_element(By.CLASS_NAME, "service")
    service_list = servicebox.find_elements(By.TAG_NAME, "img")
    svreallist = set([])
    for idx in range(len(service_list)):
        svreallist.add(service_list[idx].get_attribute("alt"))
    #print(svreallist)
    observicesetlist.append(svreallist)
    #print(len(observicesetlist), observicesetlist)
    #부가정보 세차장
    #부가정보 충전소
    #부가정보 경정비
    #부가정보 편의점
    #부가정보 24시
body2 = driver.find_element(By.ID , 'os_price3')
icount = driver.find_element(By.ID, "totCnt")
#경유 클릭
obgasol = driver.find_element(By.ID, "os_layer3")
obgasol.text
obgasol.click()
for i in range(1, int(icount.text)+1):
    g_huilist = body2.find_element(By.XPATH, """//*[@id="body3"]/tr[{}]/td[1]/a""".format(i))
    g_huilist.click()
    webdriver.ActionChains(driver).click_and_hold(up).pause(0.45).release().perform()
    #g주소
    g_box=driver.find_element(By.ID, "rd_addr")
    g_addrreal = g_box.text.split('\n')
    g_addrtext = g_addrreal[0]
    g_obloc.append(g_addrtext)
    #g상표
    g_soilbox=driver.find_element(By.ID, "poll_div_nm")
    g_soilboxtext = g_soilbox.text
    g_obicon.append(g_soilboxtext)
    #g이름
    g_namebox=driver.find_element(By.ID, "os_nm")
    #print(namebox.text)
    g_nameboxtext = g_namebox.text
    g_obname.append(nameboxtext)
    #g경유 가격
    g_huipricebox = driver.find_element(By.ID, "d047_p")
    g_huipriceboxtext = g_huipricebox.text
    g_huipriceboxtext=int(g_huipriceboxtext.replace(',', ''))
    #print(huipriceboxtext)
    g_obgprice.append(g_huipriceboxtext)
    #g부가정보 모음
    g_servicebox = driver.find_element(By.CLASS_NAME, "service")
    g_service_list = g_servicebox.find_elements(By.TAG_NAME, "img")
    g_svreallist = set([])
    for idx in range(len(g_service_list)):
        g_svreallist.add(g_service_list[idx].get_attribute("alt"))
    #print(svreallist)
    g_observicesetlist.append(g_svreallist)
huidf = pd.DataFrame(index = obname)
#huidf['obname'] = obname
huidf['obloc'] = obloc
huidf['obicon'] = obicon
huidf['obhprice'] = obhprice
huidf['observicesetlist'] = observicesetlist
g_huidf = pd.DataFrame(index = obname)
g_huidf['g_obgprice'] = g_obgprice
concat_oildf = pd.concat([huidf, g_huidf], axis = 1)
concat_oildf = concat_oildf[['obloc', 'obicon', 'obhprice', 'g_obgprice', 'observicesetlist']]
concat_oildf
for gudf in list(concat_oildf['obloc'].values):
    gudf = gudf.split(' ')
    obsi.append(gudf[0])
    obgu.append(gudf[1])
concat_oildf['si_list'] = obsi
concat_oildf['gu_list'] = obgu
concat_oildf
concat_oildf
gubyul_df.append(concat_oildf)

print(len(gubyul_df), gubyul_df)

(주)삼표에너지 정릉주유소 완료
이케이에너지(주) 강산제2주유소 완료
이케이에너지㈜ 강산주유소 완료
중앙에너비스 종암지점 완료
장삼 쌍둥이 주유소 완료
(주)성북주유소 완료
장위주유소 완료
지에스칼텍스(주) 종암주유소 완료
HD현대오일뱅크㈜직영 북악셀프주유소 완료
삼선주유소 완료
HD현대오일뱅크㈜직영 고대셀프주유소 완료
우리주유소 완료
(주)신용 구도일주유소화랑로 완료
동일주유소 완료
㈜명연에너지 효성주유소 완료
HD현대오일뱅크㈜직영 아리랑주유소 완료
1 [                                        obloc    obicon  obhprice  g_obgprice  \
(주)삼표에너지 정릉주유소           서울 성북구 보국문로 35 (정릉동)     GS칼텍스      1585        1425   
이케이에너지(주) 강산제2주유소              서울 성북구 보국문로 52  HD현대오일뱅크      1585        1425   
이케이에너지㈜ 강산주유소           서울 성북구 보문로 27 (보문동7가)     GS칼텍스      1585        1425   
중앙에너비스 종암지점                     서울 성북구 종암로 58     SK에너지      1595        1455   
장삼 쌍둥이 주유소               서울 성북구 한천로 612 (장위동)  HD현대오일뱅크      1595        1455   
(주)성북주유소                       서울 성북구 보문로 142  HD현대오일뱅크      1595        1455   
장위주유소                   서울 성북구 화랑로 110 (하월곡동)     S-OIL      1605        1455   
지에스칼텍스(주) 종암주유소          서울 성북구 종암로 145 (종암동)     GS칼텍스      1605        1465   
HD현대오일뱅크㈜직영 북

In [20]:
gubyul_df[0]

,obloc,obicon,obhprice,g_obgprice,observicesetlist,si_list,gu_list
(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1648,1519,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,1677,1529,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
HD현대오일뱅크㈜직영 산성셀프주유소,서울 강남구 헌릉로 730,HD현대오일뱅크,1677,1539,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,1697,1559,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
(주)중앙에너비스 수서지점,서울 강남구 광평로 202 (수서동),SK에너지,1717,1579,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
HD현대오일뱅크㈜직영 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),HD현대오일뱅크,1719,1594,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
지에스칼텍스㈜ 은마주유소,서울 강남구 영동대로 235,GS칼텍스,1719,1594,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
극동유화㈜ 개나리주유소,서울 강남구 언주로 423 (역삼동),S-OIL,1726,1594,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,1726,1597,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구
HD현대오일뱅크㈜직영 유진주유소,서울 강남구 논현로 152 (도곡동),HD현대오일뱅크,1729,1635,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,강남구


In [21]:
gubyul_df[15]

,obloc,obicon,obhprice,g_obgprice,observicesetlist,si_list,gu_list
(주)서울에너지 직영,서울 성동구 천호대로 324 (용답동),S-OIL,1575,1415,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
성동스카이셀프주유소,서울 성동구 천호대로 392,SK에너지,1585,1425,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
HD현대오일뱅크㈜직영 효진주유소,서울 성동구 동일로 129,HD현대오일뱅크,1599,1449,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
지에스칼텍스㈜ 도루코주유소,서울 성동구 아차산로 180,GS칼텍스,1614,1464,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
HD현대오일뱅크㈜직영 성동셀프주유소,서울 성동구 마장로 311 (마장동),HD현대오일뱅크,1617,1478,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
아이콘주유소,서울 성동구 고산자로 273 (도선동),S-OIL,1618,1487,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
동일주유소,서울 성동구 광나루로 254 (성수동2가),GS칼텍스,1649,1498,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
SK에너지(주) 군자동주유소,서울 성동구 천호대로 446 (용답동),SK에너지,1664,1519,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
SK KH에너지 성수만세주유소,서울 성동구 왕십리로 109,SK에너지,1768,1665,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구
SK세원2주유소,서울 성동구 광나루로 184 (성수동1가),SK에너지,1780,1670,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,성동구


In [22]:
gubyul_df[18]

,obloc,obicon,obhprice,g_obgprice,observicesetlist,si_list,gu_list
현대주유소,서울 양천구 남부순환로 372 (신월동),S-OIL,1582,1423,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
가로공원주유소,서울 양천구 가로공원로 165 (신월동),SK에너지,1582,1423,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
플라트(주)서호주유소,서울 양천구 남부순환로 317,GS칼텍스,1582,1423,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
지에스이앤알 직영 서울주유소,서울 양천구 신정로 236 (신정동),GS칼텍스,1599,1439,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
개나리Self주유소,서울 양천구 남부순환로 442 (신월동),SK에너지,1599,1449,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
형산석유(주)원주유소,서울 양천구 남부순환로 408,HD현대오일뱅크,1599,1449,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
삼미상사(주)신월IC셀프주유소,서울 양천구 중앙로 331 (신월동),SK에너지,1614,1449,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
양천구주유소,서울 양천구 국회대로 275 (목동),알뜰주유소,1624,1478,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
현대오일뱅크(주)직영 양천현대셀프주유소,서울 양천구 안양천로 1179 (목동),HD현대오일뱅크,1637,1478,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
목동주유소,서울 양천구 목동중앙로 125 (목동),SK에너지,1637,1497,"{경정비, 24시영업, 편의점, 세차장, 충전소}",서울,양천구
